# IMDB Lexicon

In [19]:
import tflearn
from tflearn.data_utils import to_categorical, pad_sequences
from tflearn.datasets import imdb

In [20]:
train, test, _ = imdb.load_data(path='imdb.pkl', n_words=10000, valid_portion=0.1)

In [21]:
trainX, trainY = train
testX, testY = test

In [13]:
# Flattens the dimensionality
trainX = pad_sequences(trainX, maxlen=100, value=0.)
testX = pad_sequences(testX, maxlen=100, value=0.)

# Binary encoding
trainY = to_categorical(trainY, nb_classes=2)
testY = to_categorical(testY, nb_classes=2)

In [14]:
# feed data into network - batchsize none, size 100
net = tflearn.input_data([None, 100])

# input of 10k and ouput 128
net = tflearn.embedding(net, input_dim=10000, output_dim=128)

# feed to lstm layer, drop 80%
net = tflearn.lstm(net, 128, dropout=0.8)

# cheap way of learning non linear
net = tflearn.fully_connected(net, 2, activation='softmax')

# regression operation to the input, minimize a given loss function, adam uses gradient descent
net = tflearn.regression(net, optimizer='adam', learning_rate=0.001, loss='categorical_crossentropy')

In [15]:
model = tflearn.DNN(net, tensorboard_verbose=0)
model.fit(trainX, trainY, validation_set=(testX, testY), show_metric=True, batch_size=32)

Training Step: 7039  | total loss: 0.06447 | time: 91.593s
| Adam | epoch: 010 | loss: 0.06447 - acc: 0.9840 -- iter: 22496/22500
Training Step: 7040  | total loss: 0.06057 | time: 94.107s
| Adam | epoch: 010 | loss: 0.06057 - acc: 0.9856 | val_loss: 0.82748 - val_acc: 0.8064 -- iter: 22500/22500
--


# Video Game Review Sentiment

In [55]:
import tflearn
from tflearn.data_utils import to_categorical, pad_sequences, VocabularyProcessor
from tflearn.datasets import imdb
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [56]:
dataframe = pd.read_csv('ign.csv')
dataframe.fillna("", inplace=True)
dataframe.head()

,Unnamed: 0,score_phrase,title,url,platform,score,genre,editors_choice,release_year,release_month,release_day
0,0,Amazing,LittleBigPlanet PS Vita,/games/littlebigplanet-vita/vita-98907,PlayStation Vita,9.0,Platformer,Y,2012,9,12
1,1,Amazing,LittleBigPlanet PS Vita -- Marvel Super Hero E...,/games/littlebigplanet-ps-vita-marvel-super-he...,PlayStation Vita,9.0,Platformer,Y,2012,9,12
2,2,Great,Splice: Tree of Life,/games/splice/ipad-141070,iPad,8.5,Puzzle,N,2012,9,12
3,3,Great,NHL 13,/games/nhl-13/xbox-360-128182,Xbox 360,8.5,Sports,N,2012,9,11
4,4,Great,NHL 13,/games/nhl-13/ps3-128181,PlayStation 3,8.5,Sports,N,2012,9,11


In [57]:
totalX = dataframe.title
totalY = dataframe.score

In [58]:
vocab_proc = VocabularyProcessor(15)
totalX = np.array(list(vocab_proc.fit_transform(totalX)))

In [59]:
totalY = np.array(list(totalY)) - 1
totalY = to_categorical(totalY, nb_classes=11)

In [60]:
trainX, testX, trainY, testY = train_test_split(totalX, totalY, test_size=0.1)

In [61]:
net = tflearn.input_data([None, 15])
net = tflearn.embedding(net, input_dim=10000, output_dim=256)
net = tflearn.gru(incoming=net, n_units=256, dropout=0.8, return_seq=True)
net = tflearn.gru(net, 256, dropout=0.8)
net = tflearn.fully_connected(net, 11, activation='softmax')
net = tflearn.regression(net, optimizer='adam', learning_rate=0.001, loss='categorical_crossentropy')

TypeError: Input 'split_dim' of 'Split' Op has type float32 that does not match expected type of int32.